# The Code

## Import Libraries

In [62]:
#1
from gtts import gTTS 
import os
import wikipedia
from google.cloud import translate
#from ClassText2Speech import speak_chinese
from xpinyin import Pinyin
import jieba
import jieba.posseg as pos
import jieba.analyse as ana
import sys
from importlib import *
from collections import Counter
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import time
#import matplotlib.pyplot as plt
#%matplotlib inline

# for displaying wikipedia image as output from code cell 
from PIL import Image
import requests
from io import BytesIO

from IPython.core.display import Image, display


###### Link to API Credentials
###### Google Translate API  [Setting up API Keys](https://support.google.com/googleapi/answer/6158862?hl=en )

In [63]:
#api key for google api
## you'll need to get your own API key using google cloud console and replace the path name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/db/Desktop/APIKeys/LearnChinese.json"

##### Instantiate Pinyin Generator

In [64]:
#Instantiate Frequently Used 
p = Pinyin()

### Stop Word List 
#### (add words unnecessary words here as they occur)

Its probably also a useful idea to put vocabulary words that you have already mastered into this list.

In [65]:
# List of common words that should be excluded from analysis
jieba_stop_words = ["、","。","〈","〉","《","》","一","一些","一何","一切","一则",
                    "一方面","一旦","一来","一样","一般","一转眼","七","万一","三",
                    "上","上下","下","不","不仅","不但","不光","不单","不只","不外乎",
                    "不如","不妨","不尽","不尽然","不得","不怕","不惟","不成","不拘",
                    "不料","不是","不比","不然","不特","不独","不管","不至于","不若",
                    "不论","不过","不问","与","与其","与其说","与否","与此同时","且",
                    "且不说","且说","两者","个","个别","中","临","为","为了","为什么",
                    "为何","为止","为此","为着","乃","乃至","乃至于","么","之","之一",
                    "之所以","之类","乌乎","乎","乘","九","也","也好","也罢","了","二",
                    "二来","于","于是","于是乎","云云","云尔","五","些","亦","人","人们",
                    "人家","什","什么","什么样","今","介于","仍","仍旧","从","从此","从而",
                    "他","他人","他们","他们们","以","以上","以为","以便","以免","以及",
                    "以故","以期","以来","以至","以至于","以致","们","任","任何","任凭",
                    "会","似的","但","但凡","但是","何","何以","何况","何处","何时","余外",
                    "作为","你","你们","使","使得","例如","依","依据","依照","便于","俺",
                    "俺们","倘","倘使","倘或","倘然","倘若","借","借傥然","假使","假如",
                    "假若","做","像","儿","先不先","光是","全体","全部","八","六","兮",
                    "共","关于","关于具体地说","其","其一","其中","其二","其他","其余",
                    "其它","其次","具体地说","具体说来","兼之","内","再","再其次","再则",
                    "再有","再者","再者说","再说","冒","冲","况且","几","几时","凡","凡是",
                    "凭","凭借","出于","出来","分","分别","则","则甚","别","别人","别处","别是",
                    "别的","别管","别说","到","前后","前此","前者","加之","加以","即","即令","即使",
                    "即便","即如","即或","即若","却","去","又","又及","及","及其","及至","反之",
                    "反而","反过来","反过来说","受到","另","另一方面","另外","另悉","只","只当",
                    "只怕","只是","只有","只消","只要","只限","叫","叮咚","可","可以","可是",
                    "可见","各","各个","各位","各种","各自","同","同时","后","后者","向","向使",
                    "向着","吓","吗","否则","吧","吧哒","含","吱","呀","呃","呕","呗","呜","呜呼",
                    "呢","呵","呵呵","呸","呼哧","咋","和","咚","咦","咧","咱","咱们","咳","哇",
                    "哈","哈哈","哉","哎","哎呀","哎哟","哗","哟","哦","哩","哪","哪个","哪些",
                    "哪儿","哪天","哪年","哪怕","哪样","哪边","哪里","哼","哼唷","唉","唯有","啊",
                    "啐","啥","啦","啪达","啷当","喂","喏","喔唷","喽","嗡","嗡嗡","嗬","嗯","嗳",
                    "嘎","嘎登","嘘","嘛","嘻","嘿","嘿嘿","四","因","因为","因了","因此","因着",
                    "因而","固然","在","在下","在于","地","基于","处在","多","多么","多少","大",
                    "大家","她","她们","好","如","如上","如上所述","如下","如何","如其","如同","如是",
                    "如果","如此","如若","始而","孰料","孰知","宁","宁可","宁愿","宁肯","它","它们",
                    "对","对于","对待","对方","对比","将","小","尔","尔后","尔尔","尚且","就","就是",
                    "就是了","就是说","就算","就要","尽","尽管","尽管如此","岂但","己","已","已矣","巴",
                    "巴巴","年","并","并且","庶乎","庶几","开外","开始","归","归齐","当","当地","当然",
                    "当着","彼","彼时","彼此","往","待","很","得","得了","怎","怎么","怎么办","怎么样",
                    "怎奈","怎样","总之","总的来看","总的来说","总的说来","总而言之","恰恰相反","您","惟其",
                    "慢说","我","我们","或","或则","或是","或曰","或者","截至","所","所以","所在","所幸",
                    "所有","才","才能","打","打从","把","抑或","拿","按","按照","换句话说","换言之","据",
                    "据此","接着","故","故此","故而","旁人","无","无宁","无论","既","既往","既是","既然",
                    "日","时","时候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有关",
                    "有及","有时","有的","望","朝","朝着","本","本人","本地","本着","本身","来","来着","来自",
                    "来说","极了","果然","果真","某","某个","某些","某某","根据","欤","正值","正如","正巧",
                    "正是","此","此地","此处","此外","此时","此次","此间","毋宁","每","每当","比","比及",
                    "比如","比方","没奈何","沿","沿着","漫说","焉","然则","然后","然而","照","照着","犹且",
                    "犹自","甚且","甚么","甚或","甚而","甚至","甚至于","用","用来","由","由于","由是",
                    "由此","由此可见","的","的确","的话","直到","相对而言","省得","看","眨眼","着","着呢",
                    "矣","矣乎","矣哉","离","秒","竟而","第","等","等到","等等","简言之","管","类如",
                    "紧接着","纵","纵令","纵使","纵然","经","经过","结果","给","继之","继后","继而",
                    "综上所述","罢了","者","而","而且","而况","而后","而外","而已","而是","而言","能",
                    "能否","腾","自","自个儿","自从","自各儿","自后","自家","自己","自打","自身","至",
                    "至于","至今","至若","致","般的","若","若夫","若是","若果","若非","莫不然","莫如",
                    "莫若","虽","虽则","虽然","虽说","被","要","要不","要不是","要不然","要么","要是",
                    "譬喻","譬如","让","许多","论","设使","设或","设若","诚如","诚然","该","说","说来",
                    "请","诸","诸位","诸如","谁","谁人","谁料","谁知","贼死","赖以","赶","起","起见",
                    "趁","趁着","越是","距","跟","较","较之","边","过","还","还是","还有","还要","这",
                    "这一来","这个","这么","这么些","这么样","这么点儿","这些","这会儿","这儿","这就是说",
                    "这时","这样","这次","这般","这边","这里","进而","连","连同","逐步","通过","遵循",
                    "遵照","那","那个","那么","那么些","那么样","那些","那会儿","那儿","那时","那样",
                    "那般","那边","那里","都","鄙人","鉴于","针对","阿","除","除了","除外","除开",
                    "除此之外","除非","随","随后","随时","随着","难道说","零","非","非但","非徒","非特",
                    "非独","靠","顺","顺着","首先","︿","！","＃","＄","％","＆","（","）","＊","＋",
                    "，","０","１","２","３","４","５","６","７","８","９","：",
                    "；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥","\n", " ",'借傥', '兼',
                    '前', '唷', '啪', '啷', '喔', '天', '始', '漫', '然', '特', '竟', '莫', '见', '设', '达',
                   "!" ,"#", "$", "%","&", "(",")","+", "-","/","=","'", ",", "?","他們","並且",
                   "擁有","擁有","一個","我們"]

##### english2chinese
##### Simple English to Chinese Translation Function

In [66]:
#translates English to Chinese for the initial search of wikipedia in language being learned
def english2chinese(topic, display=False):
    """ Translates a word or words from English into Traditional Mandarin"""
    # Instantiates a client
    translate_client = translate.Client()
    text = topic
    # Defines the target language
    target = 'zh-TW'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    if display == True:
        print(text_)
        return text_
    else:
        return text_

##### chinese2english()
##### Simple Chinese to English Translation Function

In [68]:
# translates Chinese to English
def chinese2english(topic):
    # Instantiates a client
    translate_client = translate.Client()
    text = topic
    # Defines the target language
    target = 'en'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    if display == True:
        print(text_)
        return text_
    else:
        return text_

##### e2c_say_anything()
##### English to Chinese With TTS

In [69]:
def e2c_say_anything():
    translate_client = translate.Client()
    text = input('What word would you like to say in Chinese(Mandarin)：')
    # Defines the target language
    target = 'zh-TW'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    
    #add pinyin
    pinyin_with_numbers = p.get_pinyin(text_, tone_marks='numbers')
    tts = gTTS(text=text_, lang='zh-TW')
    tts.save("good.mp3")
    
    print("Input: {}".format(text))
    print("Traditional Chinese Translation: {}".format(text_))
    print("Pinyin Translation with tone numbers: {}".format(pinyin_with_numbers))
    return os.system("mpg321 good.mp3")

##### c2e_say_anything
##### Chinese To English

In [70]:
def c2e_say_anything(text):
    translate_client = translate.Client()
    text = text
    # Defines the target language
    target = 'en'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    
    
    tts = gTTS(text=text_, lang='en')
    tts.save("good.mp3")
    
    print("Input: {}".format(text))
    print("Output: {}".format(text_))
    return os.system("mpg321 good.mp3")
    

##### search_wikipedia()

In [71]:
def search_wikipedia(topic, display=False,results=3):
    wikipedia.set_lang('zh-TW')
    search_phrase = english2chinese(topic)
    pages = wikipedia.search(search_phrase, results=3)
    if display == True:
        print('the pages in native language', pages)
        return pages
    else:
        return pages

##### get_content()

In [72]:
def get_content(topic):
    """Takes a topic Parameter and returns the wikipedia page that best matches that topic in Chinese"""
    wikipedia.set_lang('zh-TW')
    search_phrase = english2chinese(topic)
    wiki_page = wikipedia.page(search_phrase,preload=True)
    return wiki_page.content

##### wikipedia_page_choices()

In [73]:
def wikipedia_page_choices(topic, language='en'):
    pages = search_wikipedia(topic)
    translate_client = translate.Client()
    target_en = language
    print('Titles of wikipedia page')
    for page in pages:
            english_translation = translate_client.translate(page, target_language=target_en)
            english_text = english_translation['translatedText']
            print(english_text)
        

##### getChinese()
##### prep data for jieba text analysis

In [74]:
def getChinese(content):
    content = content.decode("utf-8") # convert content from str to unicode
    filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    content = filtrate.sub(r'', content) # remove all non-Chinese characters
    content = content.encode("utf-8") # convert unicode back to str
    return content

##### combine_wikipedia_pages

In [75]:
corpus = []
def combine_wikipedia_pages(query, suggestion=True, results=2):
    translated_query = english2chinese(query)
    titles = wikipedia.search(translated_query, results=results)
    filter('', titles)
    for title in titles:
        webpage = wikipedia.page(title)
        content = webpage.content
        image = webpage.images[::-1]
#         content = getChinese(content)
        #remove links
        #content = re.sub("http://[a-zA-z./\d]*"," ",content)
        # remove emojis
        #content = re.sub("\[.{0,12}\]"," ",content)
        #remove digits
        #content = re.sub("\d","",content)
        # remove elipses
        #content = re.sub("\.*"," ",content)
        #extract and remove all english characters
        english = re.findall("[a-z]+",content)
        content = re.sub("[a-z]+","",content)
        print("page added to content")
        f = open("myfile.txt", "a")
        f.write(str(content))
        f.close()
        picture =Image(url= str(image[0]))
        corpus.append(content)
        
    print("process finished, all content stored in myfile.txt")
    display(picture)

In [76]:
combine_wikipedia_pages('Batman')

page added to content
page added to content
process finished, all content stored in myfile.txt


In [77]:
#read in the text file
with open('myfile.txt','r')as myfile:
    data=myfile.read()

In [78]:
#segement and save 
seg_list = jieba.cut(data, cut_all=False)

In [79]:
seg_list = [str(word) for word in seg_list if not str(word) in jieba_stop_words]

In [80]:
new_text = ("".join(seg_list))

In [81]:

f=open('myfile.txt','w')
f.write(new_text)
f.close()

In [82]:
top20 = jieba.analyse.extract_tags(new_text, topK=20, withWeight=True)

In [83]:
vocab = pd.DataFrame(top20, columns=['Chinese','Weight'])

In [84]:
def get_pinyin(word, tone_marks='numbers'):
    return p.get_pinyin(word, tone_marks='numbers')

In [85]:
vocab['Pinyin']= vocab['Chinese'].apply(get_pinyin)
    

In [86]:
vocab['English'] = vocab['Chinese'].apply(chinese2english)

In [87]:
vocab

,Chinese,Weight,Pinyin,English
0,蝙蝠侠,0.426200,bian1-fu2-xia2,Batman
1,蝙蝠,0.241834,bian1-fu2,bat
2,漫画,0.099122,man4-hua4,Comic
3,DC,0.075523,DC,DC
4,罗宾,0.064301,luo1-bin1,Robin
5,布鲁斯,0.056487,bu4-lu3-si1,Bruce
6,故事,0.045367,gu4-shi4,story
7,凯恩,0.042068,kai3-en1,Kane
8,漫畫,0.041286,man4-hua4,Comic
9,连载,0.039444,lian2-zai4,Serial


##### pronounce_chinese()

In [88]:
def pronounce_chinese(text_):
    tts = gTTS(text=text_, lang='zh-TW')
    tts.save("good.mp3")
    return os.system("mpg321 good.mp3")

# Run the Below Cell Every Time

The Wikipedia (text only) content from how ever many pages you choose to pull from will be written to a text file.  The rerun function clears the file.

##### rerun()

In [89]:
def rerun():
    with open('myfile.txt', 'w') as f:
        f.write('')
        f.close()

In [90]:
rerun()

# Working MVP

# play around with the search phrase a bit

##### get_search_suggestions()

In [91]:
def get_search_suggestions():
    # user defined input
    topic = input('What would you like to search for?: ')
    # set the target language you would like to use
    # Chinese Tradtional ='zh-TW' https://ctrlq.org/code/19899-google-translate-languages
    target = 'zh-TW'
    native_language ='en'
    # Instantiates Google Translate API
    translate_client = translate.Client()
    translation = translate_client.translate(topic, target_language=target)

    # search wikipedia China
    wikipedia.set_lang(target)
    suggestions = wikipedia.search( translation['translatedText'])
    print(suggestions)
    print('try one of the following search phrases')
    #translate Chinese results into English
    for suggestion in suggestions:
        native = translate_client.translate(str(suggestion), target_language=native_language)
        print(native['translatedText'])
#     native_translation = translate_client.translate(suggestions, target_language=native_language)
#     print(native_language['translatedText'])

In [95]:
get_search_suggestions()

What would you like to search for?: tennis
['网球', '國際網球總會', '职业网球联合会', '法国网球公开赛', '澳大利亚网球公开赛', '大滿貫 (網球)', '美国网球公开赛', '夏季奧林匹克運動會網球比賽', '网球王子', '温布尔登网球锦标赛']
try one of the following search phrases
tennis
International Tennis Association
Professional tennis federation
French Open
Australian Open Tennis Championships
Grand Slam (tennis)
US Open Tennis Tournament
Summer Olympic Games Tennis Competition
tennis Prince
Wimbledon Tennis Championships


#### functions

In [96]:
# translates Chinese to English
def english2chinese(topic):
    # Instantiates a client
    translate_client = translate.Client()
    text = topic
    # Defines the target language
    target = 'zh-TW'
    # Translates some text into traditional Chinese
    translation = translate_client.translate(text, target_language=target)
    text_ = translation['translatedText']
    if display == True:
        print(text_)
        return text_
    else:
        return text_

##### application()

In [97]:
def application(results=2):
    topic=input('Type one of the above search suggestions here: ')
    combine_wikipedia_pages(topic, results=results)
    #read in the text file
    with open('myfile.txt','r')as myfile:
        data=myfile.read()
    
    #segement and save 
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = [str(word) for word in seg_list if not str(word) in jieba_stop_words]
    new_text = ("".join(seg_list))
    #overwrite myfile.txt without stopwords
    f=open('myfile.txt','w')
    f.write(new_text)
    f.close()
    #20 most important vocabulary words to know
    top20 = jieba.analyse.extract_tags(new_text, topK=20, withWeight=True)
    #store to dataframe
    vocab = pd.DataFrame(top20, columns=['Chinese','Weight'])
    #add pinyin to dataframe
    vocab['Pinyin']= vocab['Chinese'].apply(get_pinyin)
    #add english translation to dataframe
    vocab['English'] = vocab['Chinese'].apply(chinese2english)
    return vocab

#### Sanity Check

In [98]:
df = application()

Type one of the above search suggestions here: tennis
page added to content
page added to content
process finished, all content stored in myfile.txt


In [99]:
df

,Chinese,Weight,Pinyin,English
0,发球,0.242717,fa1-qiu2,Serve
1,网球,0.172912,wang3-qiu2,tennis
2,網球,0.147126,wang3-qiu2,tennis
3,比赛,0.114516,bi3-sai4,game
4,击球,0.104621,ji2-qiu2,Batting
5,反手,0.072001,fan3-shou3,Backhand
6,正手,0.064369,zheng4-shou3,Forehand
7,球网,0.053428,qiu2-wang3,Net
8,截击,0.052552,jie2-ji2,Intercept
9,比賽,0.050085,bi3-sai4,game


In [100]:
# keep only simplified or traditional 
df.drop_duplicates(subset='English',keep='first', inplace=True)

In [101]:
# Rename the columns to something I like a bit better.
df.columns = ['漢字(traditional)', 'weight', 'pinyin(numbers)', 'English']
df

,漢字(traditional),weight,pinyin(numbers),English
0,发球,0.242717,fa1-qiu2,Serve
1,网球,0.172912,wang3-qiu2,tennis
3,比赛,0.114516,bi3-sai4,game
4,击球,0.104621,ji2-qiu2,Batting
5,反手,0.072001,fan3-shou3,Backhand
6,正手,0.064369,zheng4-shou3,Forehand
7,球网,0.053428,qiu2-wang3,Net
8,截击,0.052552,jie2-ji2,Intercept
10,打出,0.048977,da3-chu1,Hit out
11,决胜局,0.047561,jue2-sheng4-ju2,Decisive game


# Bring in cc-cedict dictionary

In [102]:
dictionary = pd.read_csv('cedict.csv',sep='|', skiprows=0)

In [103]:
dictionary.columns = ["漢字(traditional)", "汉字(simplified)", "pinyin(marks)", "English2"]

In [104]:
df2= pd.merge(df, dictionary, on="漢字(traditional)", how='left')

In [105]:
df2.columns

Index(['漢字(traditional)', 'weight', 'pinyin(numbers)', 'English',
       '汉字(simplified)', 'pinyin(marks)', 'English2'],
      dtype='object')

In [112]:
# rearrange the columns in a way that makes sense
df2 = df2[["English","English2","weight","漢字(traditional)","汉字(simplified)","pinyin(marks)","pinyin(numbers)"]]

In [113]:
df2

,English,English2,weight,漢字(traditional),汉字(simplified),pinyin(marks),pinyin(numbers)
0,Serve,NaN,0.242717,发球,NaN,NaN,fa1-qiu2
1,tennis,NaN,0.172912,网球,NaN,NaN,wang3-qiu2
2,game,NaN,0.114516,比赛,NaN,NaN,bi3-sai4
3,Batting,NaN,0.104621,击球,NaN,NaN,ji2-qiu2
4,Backhand,to turn a hand over; to put one's hand behind ...,0.072001,反手,NaN,fǎn shǒu,fan3-shou3
5,Forehand,NaN,0.064369,正手,NaN,NaN,zheng4-shou3
6,Net,NaN,0.053428,球网,NaN,NaN,qiu2-wang3
7,Intercept,NaN,0.052552,截击,NaN,NaN,jie2-ji2
8,Hit out,NaN,0.048977,打出,NaN,NaN,da3-chu1
9,Decisive game,NaN,0.047561,决胜局,NaN,NaN,jue2-sheng4-ju2


https://en.wiktionary.org/wiki/Appendix:Mandarin_Frequency_lists

In [108]:
most_common_words = pd.read_csv('1000words.csv', index_col=0)

In [109]:
most_common_words.sample(5)

,Simplified,Pinyin,Meaning
Traditional,,,
張,张,zhāng,m.[general]
死,死,sǐ,die
要求,要求,yāoqiú,"request, demand"
那麼,那么,nàmo,so
產業,产业,chǎnyè,"estate, manufacture, industry, domain, manufac..."


# Study Section

## [Review the Tones](https://www.youtube.com/watch?v=4ZB9-vZFesk)

# Review Most Frequent Words

In [110]:
most_common_words.sample(5)

,Simplified,Pinyin,Meaning
Traditional,,,
豐富,丰富,fēngfù,"rich, abundant, plentiful"
空間,空间,kōngjiān,"elbow room, allowance, margin, roomage, open a..."
電子,电子,diànzǐ,electron
資源,资源,zīyuán,"resource, natural resources, mine, resources"
份,份,fèn,m.[general]


## New Vocabulary Words

In [114]:
df2

,English,English2,weight,漢字(traditional),汉字(simplified),pinyin(marks),pinyin(numbers)
0,Serve,NaN,0.242717,发球,NaN,NaN,fa1-qiu2
1,tennis,NaN,0.172912,网球,NaN,NaN,wang3-qiu2
2,game,NaN,0.114516,比赛,NaN,NaN,bi3-sai4
3,Batting,NaN,0.104621,击球,NaN,NaN,ji2-qiu2
4,Backhand,to turn a hand over; to put one's hand behind ...,0.072001,反手,NaN,fǎn shǒu,fan3-shou3
5,Forehand,NaN,0.064369,正手,NaN,NaN,zheng4-shou3
6,Net,NaN,0.053428,球网,NaN,NaN,qiu2-wang3
7,Intercept,NaN,0.052552,截击,NaN,NaN,jie2-ji2
8,Hit out,NaN,0.048977,打出,NaN,NaN,da3-chu1
9,Decisive game,NaN,0.047561,决胜局,NaN,NaN,jue2-sheng4-ju2


In [68]:
df.columns = ['漢字(traditional)', 'weight', 'pinyin(numbers)', 'English']

# keep only simplified or traditional 
df.drop_duplicates(subset='English',keep='first', inplace=True)

df2= pd.merge(df, dictionary, on="漢字(traditional)", how='left')
df2 = df2[["漢字(traditional)","汉字(simplified)","English","English2","pinyin(marks)","pinyin(numbers)"]]
df2

,漢字(traditional),汉字(simplified),English,English2,pinyin(marks),pinyin(numbers)
0,葉問,叶问,Ip man,"Yip Man (1893-1972), martial arts practitioner...",Yè Wèn,ye4-wen4
1,詠春拳,咏春拳,Wing Chun,"Yongchun - ""Singing Spring Fist"" (Chinese mart...",yǒng chūn quán,yong3-chun1-quan2
2,洪震南,NaN,Hong Zhennan,NaN,NaN,hong2-zhen4-nan2
3,香港,NaN,Hong Kong,Hong Kong.,Xiāng gǎng,xiang1-gang3
4,梁相,NaN,Beam phase,NaN,NaN,liang2-xiang1
5,佛山,NaN,Foshan,Foshan prefecture-level city in Guangdong.,Fó shān,fo2-shan1
6,吳仲素,NaN,Wu Zhongsu,NaN,NaN,wu2-zhong4-su4
7,武術,武术,Martial arts,military skill or technique (in former times);...,wǔ shù,wu3-shu4
8,當時,当时,then,then; at that time; while.,dāng shí,dang1-shi2
9,當時,当时,then,at once; right away.,dàng shí,dang1-shi2


## Practice Pronouncing The Words

comment out or remove print row["漢字(traditional)"] and/row["pinyin(marks)"] if you do not want the chinese characters to print out in the practice

In [118]:
for index,row in df.iterrows():
    print(row['English'], row["漢字(traditional)"], row["pinyin(numbers)"])
    #time.sleep(10)
    pronounce_chinese(row['漢字(traditional)'])

Serve 发球 fa1-qiu2
tennis 网球 wang3-qiu2
game 比赛 bi3-sai4
Batting 击球 ji2-qiu2
Backhand 反手 fan3-shou3
Forehand 正手 zheng4-shou3
Net 球网 qiu2-wang3
Intercept 截击 jie2-ji2
Hit out 打出 da3-chu1
Decisive game 决胜局 jue2-sheng4-ju2
Returning the ball 回球 hui2-qiu2
Handle 拍柄 pai1-bing3
win 赢得 ying2-de2
racket 球拍 qiu2-pai1
Receiving the ball 接发球 jie1-fa1-qiu2
depth 深度 shen1-du4
Player 球员 qiu2-yuan2


Put individual words in here to hear them one at a time

In [120]:
pronounce_chinese('生')

0

In [121]:
e2c_say_anything()

What word would you like to say in Chinese(Mandarin)：Would you like to play tennis?
Input: Would you like to play tennis?
Traditional Chinese Translation: 你想打網球嗎？
Pinyin Translation with tone numbers: ni3-xiang3-da3-wang3-qiu2-ma5-？


0

In [83]:
e2s_say_anything()

What word would you like to say in Chinese(Mandarin)：武術
Input: 武術
Traditional Chinese Translation: 武術
Pinyin Translation with tone numbers: wu3-shu4


0

In [82]:
c2e_say_anything('武術')

Input: 武術
Output: Martial arts


0